# TPB708 Programación de aplicaciones en sistemas de información geográfica
## Proyecto 02 - Visualización de tablas, gráficos y mapas

### Módulos

Instalación de módulos

```
conda install -c conda-forge chart-studio
```

### Carga de módulos

In [2]:
from __future__ import print_function

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from plotly.offline import iplot

import chart_studio.plotly as py

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import folium
import shapely

from shapely.geometry import shape
from collections import OrderedDict

import fiona

import numpy as np



### Datos de casos de COVID-19 en Costa Rica

In [3]:
# Carga en dataframes de casos positivos, activos, recuperados y fallecidos para todas las fechas
positivos_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_POSITIVOS.csv", 
                           encoding = "latin")
activos_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_ACTIVOS.csv", 
                           encoding = "latin")
recuperados_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_RECUP.csv", 
                           encoding = "latin")
fallecidos_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_FALLECIDOS.csv", 
                            encoding = "latin")

# Carga en un dataframe de casos positivos, activos, recuperados y fallecidos para la última fecha
ultima_fecha_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_ULTIMA_FECHA.csv", 
                               encoding = "latin")

**Ejercicio 01 (20%):** Mediante la función create_table() de plotly.figure_factory, despliegue las cantidades de casos (positivos, recuperados, activos y fallecidos) para los cantones de la provincia de Cartago, para la última fecha disponible. Debe desplegarse también el nombre de la provincia y del cantón.

In [5]:
# Despliegue de tabla de casos positivos mediante la función create_table() de plotly.figure_factory

ultima_fecha_tabla = ff.create_table(ultima_fecha_df
[ultima_fecha_df['provincia'] == 'Cartago'])
iplot(ultima_fecha_tabla)




**Ejercicio 02 (20%):** Mediante la función Table() de plotly.graph_objects, despliegue las cantidades de casos positivos correspondientes a la última semana de octubre de 2020, para los cantones de la provincia de Heredia.

In [7]:
data = positivos_df [['provincia', 'canton', '25/10/2020', '26/10/2020', '27/10/2020', '28/10/2020', '29/10/2020', '30/10/2020','31/10/2020']]
chp = data [positivos_df ['provincia'] == 'Heredia']

chp.columns = chp.columns.str.replace ('25/10/2020', 'Domingo')
chp.columns = chp.columns.str.replace ('26/10/2020', 'Lunes')
chp.columns = chp.columns.str.replace ('27/10/2020', 'Martes')
chp.columns = chp.columns.str.replace ('28/10/2020', 'Miercoles')
chp.columns = chp.columns.str.replace ('29/10/2020', 'Jueves')
chp.columns = chp.columns.str.replace ('30/10/2020', 'Viernes')
chp.columns = chp.columns.str.replace ('31/10/2020', 'Sabado')


# Despliegue de tabla de casos positivos mediante la función Table() de plotly.graph_objects

fig = go.Figure(data=[go.Table(
    header=dict(values=['Provincia', 'Canton', '25/10/2020', '26/10/2020', '27/10/2020', '28/10/2020', '29/10/2020', '30/10/2020','31/10/2020'],
                fill_color= 'slategrey',
                line_color='darkslategray',
                font=dict(color='black', size=16),
                align=['left','center']),
             
    cells=dict(values=[chp.provincia, chp.canton, chp.Domingo, chp.Lunes, chp.Martes, chp.Miercoles, chp.Jueves, chp.Viernes, chp.Sabado],
                fill_color = 'lightcyan',
                line_color='darkolivegreen',
                font = dict(color = 'darkslategray', size = 14),
                align=['center']))
           

])
fig.show()

**Ejercicio 03 (25%)**: Mediante la función bar() de plotly.express (https://plotly.com/python-api-reference/generated/plotly.express.bar.html), genere un gráfico de barras que, de manera similar al gráfico de dispersión anterior, muestre las cantidades de casos positivos en los cantones de Costa Rica con mayor cantidad de casos.

El gráfico debe mostrar claramente:

* Un título.
* Las etiquetas en los ejes x e y.
* Los nombres de los cantones.
* La cantidad de casos para cada cantón.

In [19]:
ultima_fecha_ordenado_df = ultima_fecha_df.sort_values("positivos", ascending=False)

fig = px.bar (ultima_fecha_ordenado_df.head(15),
                 x="canton", y="positivos",
                 color="positivos",
                 hover_name="canton",
                 orientation= "v",
                 barmode= 'relative',
                 color_continuous_scale= 'puor',  
                 title= "Cantones de Costa Rica con mayor cantidad en casos positivos de COVID-19 durante el 2020")


fig.show()

In [ ]:
# Mapa de coropletas de casos por tipo en cantones

def mapa_casos_x_tipo(tipo_casos):
    # GeoJSON de cantones
    geo_cantones = r'https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/delimitacion-territorial-administrativa/cr/ign/cr_limite_cantonal_ign_wgs84.geojson'

    # Mapa base
    m = folium.Map(
        location=[10, -84], 
        zoom_start=7, 
        tiles='openstreetmap')
    
    fill_color = "Blues"
    if tipo_casos == "activos":
        fill_color = "Reds"
    elif tipo_casos == "recuperados":
        fill_color = "Greens"
    elif tipo_casos == "fallecidos":
        fill_color = "Greys"


    # Capa de coropletas coloreada de acuerdo con el campo de casos positivos.
    # El enlace entre los datos geoespaciales y tabulares se realiza a través del campo "cod_canton" del dataframe
    # y el campo del mismo nombre en el archivo GeoJSON.
    m.choropleth(
        geo_data=geo_cantones,
        data=ultima_fecha_df,
        columns=['cod_canton', tipo_casos],
        key_on='feature.properties.cod_canton',
        fill_color=fill_color, 
        fill_opacity=1, 
        line_opacity=1,
        legend_name='Casos ' + tipo_casos,
        smooth_factor=0)


    # Despliegue del mapa
    return m
    
mapa_casos_x_tipo("activos") 
# el parámetro de entrada puede tener los valores "positivos", "activos", "recuperados" o "fallecidos".

**Ejercicio 04 (35%):** modifique el mapa de coropletas del ejemplo anterior de manera que la cantidad de casos en cada cantón se muestre mediante mediante símbolos proporcionales (ej. https://coronavirus.jhu.edu/map.html). Al hacer clic sobre el símbolo de un cantón, debe mostrarse el nombre del cantón y la cantidad de casos.

In [ ]:
# Mapa de coropletas de casos por tipo en cantones

def mapa_casos_x_tipo(tipo_casos):
    # GeoJSON de cantones
    cantones = gpd.read_file('https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/delimitacion-territorial-administrativa/cr/ign/cr_limite_cantonal_ign_wgs84.geojson')
    geo_cantones = cantones.copy()
    esquema_centroides = {
    'geometry':'Point'
   
    
    
    
    